# **Cleaning master data**

Here I create the master dataset using World Bank APIs, FMI and Economic Complexity data.

In [89]:
import wbgapi as wb # World bank
import pandas as pd
from weo import download, WEO # IMF data
import requests, io
import rdata   # or pyreadr
import os

### Variable selection

In [115]:
# World Bank
wb_variables = [

#    'NY.GDP.PCAP.CD', # GDP per capita (current US$) # Note: I download it in the IMF request
#    'NY.GNP.ATLS.CD', # GNI, Atlas method (current US$)

    # Resource Intensity
    
    'NY.GDP.TOTL.RT.ZS', # Total natural resources rents (% of GDP)

    'NV.IND.MANF.ZS',        # Manufacturing, value added (% of GDP)
    'NV.IND.TOTL.ZS',        # Industry (including construction), value added (% of GDP)
    'TX.VAL.TECH.MF.ZS',     # High-technology exports (% of manufactured exports)
    'NV.AGR.TOTL.ZS',        # Agriculture, forestry, and fishing, value added (% of GDP)
    'NV.SRV.TOTL.ZS',        # Services, value added (% of GDP)
    'NY.GDP.MINR.RT.ZS',     # Mineral rents (% of GDP)
    'NY.GDP.NGAS.RT.ZS',     # Natural gas rents (% of GDP)
    'NY.GDP.PETR.RT.ZS',      # Oil rents (% of GDP)
    
    'NY.ADJ.SVNG.CD', #  Adjusted savings: total (current US$)
    'NY.ADJ.DRES.GN.ZS', # Adjusted savings: natural resources depletion (% of GNI)    
    'IQ.CPA.HRES.XQ', # CPIA building human resources rating (1=low to 6=high)',
    'IQ.CPA.FINQ.XQ', # 'CPIA quality of budgetary and financial management rating (1=low to 6=high)',
    'IQ.CPA.TRAN.XQ', #'CPIA transparency, accountability and corruption in the public sector rating (1=low to 6=high)',
    'DT.DOD.DIMF.CD' # Use of IMF credit (DOD, current US$)
    ]

# IMF 
imf_variables = [
    ("Gross domestic product per capita, constant prices",
     "Purchasing power parity; 2017 international dollar"),
    
    ("General government revenue",
     "Percent of GDP"),
]

imf_icsd_variables = [
    'P51G_S13_Q_POGDP_PT.A' # Gross fixed capital formation, General government, Constant prices, Percent of GDP
    'P51G_PS_Q_POGDP_PT.A' # Gross fixed capital formation, Private sector, Constant prices, Percent of GDP
    'P51G_PUPVT_Q_POGDP_PT.A' # Gross fixed capital formation, Public private partnership, Constant prices, Percent of GDP
]


# Economic Complexity
eci_variables = [
    'eci'
 ]

# V Democracy
vdem_variables = [
    # High-Level Democracy Indices
    'v2x_polyarchy', # Electoral democracy index
    'v2x_libdem', # Liberal democracy index
    'v2x_partipdem', # Participatory democracy index
    'v2x_delibdem', # Deliberative democracy index
    'v2x_egaldem', # Egalitarian democracy index
    
    'v2xnp_client', #  Clientelism Index (D)  
    'v2x_corr', # Political corruption index (D) 
    'v2x_rule', # Rule of law index (D) 
    'v2x_accountability', # Accountability index
    'v2xcl_prpty', # Property rights (D) 
    'e_wbgi_pve', # Political stability — estimate (E) 
    'e_civil_war' # Civil war (E)

]

# Penn World Table - FLAG THESE ARE IN NATIONAL CURRENCY
pwt_variables = [
    'hc', # Human capital index
    'cn', # Capital stock (national accounts prices) Unit: constant local currency (real terms)
    'ctfp', # TFP level (constant national prices) Unit: index
    'cwtfp', # Welfare-relevant TFP    
    'csh_c', # Share of consumption in GDP
    "csh_i", # Share of investment in GDP
    "csh_g", # Share of government spending in GDP
    "delta" # Capital depreciation rate
]


### World Bank API

In [91]:
def download_wb_indicators(indicators, start_year, end_year):
    final_rows = []

    # Get all real countries
    economies = [c['id'] for c in wb.economy.list() if not c.get("aggregate", False)]
    
    for indicator in indicators:
        print(f"Downloading {indicator} ...")
        raw = wb.data.fetch(indicator, economy=economies, time=range(start_year, end_year + 1))
        
        for row in raw:
            iso = row.get("economy")
            year = int(row.get("time").replace("YR", ""))
            value = row.get("value")
            
            if iso is None or value is None:
                continue

            final_rows.append({
                "Country Code": iso,
                "Year": year,
                "Variable": indicator,
                "Value": value
            })

    df = pd.DataFrame(final_rows)
    return df

wb_df = download_wb_indicators(wb_variables, start_year=1990, end_year=2024)

In [92]:
# Get all economies (countries + aggregates)
all_economies = wb.economy.list()

# Filter only real countries (exclude aggregates)
countries = [c for c in all_economies if not c.get("aggregate", False)]

# Build a dataframe
country_names = pd.DataFrame({
    "Country Code": [c["id"] for c in countries],
    "Country Name": [c["value"] for c in countries]
})

country_names

,Country Code,Country Name
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,ALB,Albania
4,AND,Andorra
...,...,...
212,XKX,Kosovo
213,YEM,"Yemen, Rep."
214,ZAF,South Africa
215,ZMB,Zambia


#### IMF

In [93]:
path, _ = download(2024, "Apr")  # or adjust to whichever vintage you want
w = WEO(path)
frames = []
    
frames = []
for subj, unit in imf_variables:
    df = w.get(subj, unit).reset_index().rename(columns={"index": "COUNTRY"})
    df_long = df.melt(id_vars="COUNTRY", var_name="YEAR", value_name="VALUE")
    df_long["INDICATOR"] = subj
    frames.append(df_long)

imf_df = pd.concat(frames, ignore_index=True)
print(imf_df.head())

# Remode Weo data from code folder
os.remove("weo_2024_1.csv")

imf_df = imf_df.rename(columns={
    'COUNTRY': 'Year',
    'YEAR': 'Country Code',
    'INDICATOR': 'Variable',
    'VALUE': 'Value'
})
imf_df

weo_2024_1.csv 19.2Mb
Downloaded 2024-Apr WEO dataset
  COUNTRY YEAR  VALUE                                          INDICATOR
0    1980  AFG    NaN  Gross domestic product per capita, constant pr...
1    1981  AFG    NaN  Gross domestic product per capita, constant pr...
2    1982  AFG    NaN  Gross domestic product per capita, constant pr...
3    1983  AFG    NaN  Gross domestic product per capita, constant pr...
4    1984  AFG    NaN  Gross domestic product per capita, constant pr...


,Year,Country Code,Value,Variable
0,1980,AFG,NaN,"Gross domestic product per capita, constant pr..."
1,1981,AFG,NaN,"Gross domestic product per capita, constant pr..."
2,1982,AFG,NaN,"Gross domestic product per capita, constant pr..."
3,1983,AFG,NaN,"Gross domestic product per capita, constant pr..."
4,1984,AFG,NaN,"Gross domestic product per capita, constant pr..."
...,...,...,...,...
19595,2025,ZWE,16.465,General government revenue
19596,2026,ZWE,16.462,General government revenue
19597,2027,ZWE,16.380,General government revenue
19598,2028,ZWE,16.381,General government revenue


#### IMF ICSD

In [ ]:
# ECI
icsd_df = (
    pd.read_csv('https://raw.githubusercontent.com/AyaanTigdikar/Capstone/refs/heads/main/rawdata/growth_proj_eci_rankings.csv') # Note: If error, update the raw link in here: https://github.com/AyaanTigdikar/Capstone/blob/main/rawdata/growth_proj_eci_rankings.csv
      .rename(columns={'country_iso3_code': 'country_code', 'eci_hs92': 'eci'})
      .drop(columns=['eci_rank_hs92'])
)

# Country codes
continent_labels = pd.read_csv('../../data/location_group_member.csv')

eci_clean = (
    eci_df
      .merge(
          continent_labels[continent_labels['group_type'] == 'continent'][['group_name', 'country_id']],
          on='country_id',
          how='left'
      )
      .rename(columns={'group_name': 'continent'})
      .drop_duplicates(subset='country_code', keep='first')
)

eci_df['Variable'] = 'Economic Complexity'

eci_df = eci_df.rename(columns={
  'country_code': 'Country Code',
  'year': 'Year',
  'eci': 'Value'
})

eci_df = eci_df[['Country Code', 'Year', 'Variable' , 'Value']]

ConnectTimeout: HTTPSConnectionPool(host='dataservices.imf.org', port=443): Max retries exceeded with url: /REST/SDMX_JSON.svc/CompactData/ICSD/COUNTRY.*.P51G_S13_Q_POGDP_PT.AP51G_PS_Q_POGDP_PT.AP51G_PUPVT_Q_POGDP_PT.A.A?startPeriod=1990&endPeriod=2019 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000234201CAAD0>, 'Connection to dataservices.imf.org timed out. (connect timeout=None)'))

#### Economic Complexity

In [94]:
# ECI
eci_df = (
    pd.read_csv('https://raw.githubusercontent.com/AyaanTigdikar/Capstone/refs/heads/main/rawdata/growth_proj_eci_rankings.csv') # Note: If error, update the raw link in here: https://github.com/AyaanTigdikar/Capstone/blob/main/rawdata/growth_proj_eci_rankings.csv
      .rename(columns={'country_iso3_code': 'country_code', 'eci_hs92': 'eci'})
      .drop(columns=['eci_rank_hs92'])
)

# Country codes
continent_labels = pd.read_csv('../../data/location_group_member.csv')

eci_clean = (
    eci_df
      .merge(
          continent_labels[continent_labels['group_type'] == 'continent'][['group_name', 'country_id']],
          on='country_id',
          how='left'
      )
      .rename(columns={'group_name': 'continent'})
      .drop_duplicates(subset='country_code', keep='first')
)

eci_df['Variable'] = 'Economic Complexity'

eci_df = eci_df.rename(columns={
  'country_code': 'Country Code',
  'year': 'Year',
  'eci': 'Value'
})

eci_df = eci_df[['Country Code', 'Year', 'Variable' , 'Value']]

### V democracy

In [95]:
# 1. Download the RData from GitHub “raw” URL
url = ("https://raw.githubusercontent.com/vdeminstitute/vdemdata/master/data/vdem.RData")
resp = requests.get(url)
resp.raise_for_status()  # check download succeeded

with open("vdem.RData", "wb") as f:
    f.write(resp.content)

# Load the .RData file
vdem_r = rdata.read_rda("vdem.RData")

# Read R data as a df
vdem = vdem_r.get("vdem")

# Remode RData from code folder
os.remove("vdem.RData")
    
var_list = ['country_name', 'country_text_id', 'year'] + vdem_variables

vdem = vdem[var_list]

vdem = vdem.rename(columns={
                   'country_name': 'Country Name',
                   'country_text_id': 'Country Code',
                   'year': 'Year',
})  
    
vdem_df = pd.melt(
    vdem,
    id_vars=['Country Code','Year'],
    value_vars= vdem_variables,
    var_name='Variable',
    value_name='Value'
)

vdem_df = vdem_df[vdem_df['Year'] >= 1990]

vdem_df

c:\Users\iorueta\AppData\Local\Programs\Python\Python313\Lib\site-packages\rdata\conversion\_conversion.py:900: UserWarning: Missing constructor for R class "Date". The underlying R object is returned instead.
  warnings.warn(


,Country Code,Year,Variable,Value
201,MEX,1990.0,v2x_polyarchy,0.389
202,MEX,1991.0,v2x_polyarchy,0.412
203,MEX,1992.0,v2x_polyarchy,0.437
204,MEX,1993.0,v2x_polyarchy,0.447
205,MEX,1994.0,v2x_polyarchy,0.470
...,...,...,...,...
333552,ZZB,2020.0,e_civil_war,NaN
333553,ZZB,2021.0,e_civil_war,NaN
333554,ZZB,2022.0,e_civil_war,NaN
333555,ZZB,2023.0,e_civil_war,NaN


### Penn World Table

In [96]:
# Penn World Table 11.0
url = "https://raw.githubusercontent.com/AyaanTigdikar/Capstone/main/rawdata/pwt110.xlsx"
pwt_df = (pd.read_excel(url, engine="openpyxl", sheet_name='Data')
          .rename(columns={'countrycode': 'Country Code', 
                           'country': 'Country Name', 
                           'year': 'Year',
                           'currency_unit': 'Currency'}))

pwt_df = pwt_df[['Country Code','Country Name','Year', 'Currency'] + pwt_variables]

pwt_df = pwt_df.melt(
    id_vars=['Country Code', 'Currency', 'Year'],
    value_vars=pwt_variables,
    var_name='Variable',
    value_name='Value'
)

pwt_df


,Country Code,Currency,Year,Variable,Value
0,ABW,Aruban Guilder,1950,hc,NaN
1,ABW,Aruban Guilder,1951,hc,NaN
2,ABW,Aruban Guilder,1952,hc,NaN
3,ABW,Aruban Guilder,1953,hc,NaN
4,ABW,Aruban Guilder,1954,hc,NaN
...,...,...,...,...,...
109515,ZWE,US Dollar,2019,delta,0.059923
109516,ZWE,US Dollar,2020,delta,0.060208
109517,ZWE,US Dollar,2021,delta,0.060136
109518,ZWE,US Dollar,2022,delta,0.059788


## Merging and final cleaning

In [ ]:
final_df = pd.concat([wb_df, eci_df, imf_df, vdem_df, pwt_df])

# Variable names
rename_map = {
    # WBI
    "NV.IND.MANF.ZS": "Manufacturing",
    "NV.IND.TOTL.ZS": "Industry",
    "TX.VAL.TECH.MF.ZS": "High-tech exports",
    "NV.AGR.TOTL.ZS": "Agriculture",
    "NV.SRV.TOTL.ZS": "Services",
    'NY.GDP.TOTL.RT.ZS': 'Total natural resources rents (% of GDP)', # Total natural resources rents (% of GDP)
    'NY.GDP.MINR.RT.ZS': 'Mineral rents (% of GDP)',     # Mineral rents (% of GDP)
    'NY.GDP.NGAS.RT.ZS': 'Natural gas rents (% of GDP)',     # Natural gas rents (% of GDP)
    'NY.GDP.PETR.RT.ZS': 'Oil rents (% of GDP)',      # Oil rents (% of GDP)
    'NY.ADJ.SVNG.CD': 'Adjusted savings: total (current US$)',
    'NY.ADJ.DRES.GN.ZS': 'Adjusted savings: natural resources depletion (% of GNI)',
    'IQ.CPA.HRES.XQ': 'CPIA building human resources rating (1=low to 6=high)',
    'IQ.CPA.FINQ.XQ': 'CPIA quality of budgetary and financial management rating (1=low to 6=high)',
    'IQ.CPA.TRAN.XQ': 'CPIA transparency, accountability and corruption in the public sector rating (1=low to 6=high)',
    'DT.DOD.DIMF.CD': 'Use of IMF credit (DOD, current US$)',
    
    # ECI
    "Economic Complexity": "Economic Complexity Index",

    # FMI
    "Gross domestic product per capita, constant prices": "GDP per capita (constant prices, PPP)",
    "General government revenue": "Government revenue",

    # VDEM
    'v2x_polyarchy': 'electoral_dem', # Electoral democracy index
    'v2x_libdem': 'liberal_dem', # Liberal democracy index
    'v2x_partipdem': 'participatory_dem', # Participatory democracy index
    'v2x_delibdem': 'deliberative_dem', # Deliberative democracy index
    'v2x_egaldem': 'egalitarian_dem', # Egalitarian democracy index
    
    'v2xnp_client': 'Clientelism index',  
    'v2x_corr': 'Political corruption index', 
    'v2x_rule': 'Rule of law index', 
    'v2x_accountability': 'Accountability index',
    'v2xcl_prpty': 'Property rights', 
    'e_wbgi_pve': 'Political stability — estimate', 
    'e_civil_war': 'Civil war',
    
    # PWT
    'hc': 'Human capital index',
    'cn': 'Capital stock (national accounts prices)',
    'ctfp': 'TFP level (constant national prices)',
    'cwtfp': 'Welfare-relevant TFP',   
    'csh_c': 'Share of consumption in GDP',
    "csh_i":  'Share of investment in GDP',
    "csh_g": 'Share of government spending in GDP',
    "delta": 'Capital depreciation rate'

}

final_df["Variable"] = final_df["Variable"].replace(rename_map)

# Countries names
final_df = final_df.merge(country_names, how='left', on='Country Code')

# Filter period
# If Year is a pandas Period, extract the year as int
final_df['Year'] = final_df['Year'].apply(lambda x: x.year if hasattr(x, 'year') else int(x))

final_df = final_df[(final_df['Year'] >= 1990) & (final_df['Year'] <= 2024)]


In [ ]:
final_df['Variable'].nunique() # 38 unique variables. That's correct.

38

#### To wide

In [106]:
final_df_wide = final_df.pivot(
    index=['Country Code', 'Country Name', 'Year'],
    columns='Variable',
    values='Value'
).reset_index()

In [108]:
final_df_wide

Variable,Country Code,Country Name,Year,Accountability index,Adjusted savings: natural resources depletion (% of GNI),Adjusted savings: total (current US$),Agriculture,CPIA building human resources rating (1=low to 6=high),CPIA quality of budgetary and financial management rating (1=low to 6=high),"CPIA transparency, accountability and corruption in the public sector rating (1=low to 6=high)",...,Share of investment in GDP,TFP level (constant national prices),Total natural resources rents (% of GDP),"Use of IMF credit (DOD, current US$)",Welfare-relevant TFP,deliberative_dem,egalitarian_dem,electoral_dem,liberal_dem,participatory_dem
0,ABW,Aruba,1990,NaN,0.001568,NaN,NaN,NaN,NaN,NaN,...,0.337052,NaN,0.001552,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,Aruba,1991,NaN,0.001649,NaN,NaN,NaN,NaN,NaN,...,0.405814,NaN,0.001634,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABW,Aruba,1992,NaN,0.001393,NaN,NaN,NaN,NaN,NaN,...,0.297919,NaN,0.001379,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABW,Aruba,1993,NaN,0.000941,NaN,NaN,NaN,NaN,NaN,...,0.322841,NaN,0.000925,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABW,Aruba,1994,NaN,0.000953,NaN,NaN,NaN,NaN,NaN,...,0.306251,NaN,0.000937,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7673,ZZB,NaN,2020,0.048,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.263,0.249,0.271,0.219,0.174
7674,ZZB,NaN,2021,0.111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.273,0.256,0.285,0.232,0.180
7675,ZZB,NaN,2022,0.171,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.278,0.265,0.294,0.240,0.182
7676,ZZB,NaN,2023,0.188,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.281,0.262,0.298,0.242,0.186


In [ ]:
final_df_wide.shape # Country name + Country Code + Year + 38 variables.

(7678, 41)

## Saving data

In [101]:
final_df.to_csv('../workingdata/master_data_long.csv')

final_df_wide.to_csv('../workingdata/master_data_wide.csv')